In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

# Load data
df_prodi = pd.read_csv('daftar_prodi.csv')
df_univ = pd.read_csv('daftar_universitas.csv')

# Merge nama universitas ke prodi
df = df_prodi.merge(df_univ[['kode_univ', 'nama_univ']], on='kode_univ', how='left')

training_rows = []

for tahun in range(2018, 2023):  # 2018 s/d 2022
    target_tahun = tahun + 1
    if f'peminat_{target_tahun}' in df.columns and f'daya_tampung_{target_tahun}' in df.columns:
        peluang_target = df[f'daya_tampung_{target_tahun}'] / df[f'peminat_{target_tahun}'].replace(0, np.nan)
        temp = pd.DataFrame({
            'peminat': df[f'peminat_{tahun}'],
            'daya_tampung': df[f'daya_tampung_{tahun}'],
            'peluang_target': peluang_target
        })
        training_rows.append(temp)

df_training = pd.concat(training_rows, ignore_index=True).dropna().reset_index(drop=True)

X_train = df_training[['peminat', 'daya_tampung']].values
y_train = df_training['peluang_target'].values

model = LinearRegression()
model.fit(X_train, y_train)

# Perbaikan: gunakan .values saat predict agar input sesuai
fitur_2022 = df[['peminat_2022', 'daya_tampung_2022']].fillna(0).values
df['prediksi_peluang_2023'] = model.predict(fitur_2022)

scaler = MinMaxScaler()
df['prediksi_scaled_2023'] = scaler.fit_transform(df[['prediksi_peluang_2023']])

def rekomendasi_berdasarkan_prodi(nama_prodi_input, top_n=10):
    mask = df['nama_prodi'].str.lower().str.contains(nama_prodi_input.lower())
    if not mask.any():
        return f"Program studi '{nama_prodi_input}' tidak ditemukan."

    subdf = df[mask].copy()
    subdf_sorted = subdf.sort_values(by='prediksi_scaled_2023', ascending=False).head(top_n)

    rekom_list = []
    for i, row in subdf_sorted.iterrows():
        rekom_list.append({
            'rank': len(rekom_list) + 1,
            'nama_prodi': row['nama_prodi'],
            'nama_univ': row['nama_univ'],
            'prediksi_peluang_2023': row['prediksi_peluang_2023'],
            'prediksi_scaled_2023': row['prediksi_scaled_2023']
        })
    return rekom_list

def rekomendasi_berdasarkan_universitas(nama_univ_input, top_n=10):
    mask = df['nama_univ'].str.lower().str.contains(nama_univ_input.lower())
    if not mask.any():
        return f"Universitas '{nama_univ_input}' tidak ditemukan."

    subdf = df[mask].copy()
    subdf_sorted = subdf.sort_values(by='prediksi_scaled_2023', ascending=False).head(top_n)

    rekom_list = []
    for i, row in subdf_sorted.iterrows():
        rekom_list.append({
            'rank': len(rekom_list) + 1,
            'nama_prodi': row['nama_prodi'],
            'nama_univ': row['nama_univ'],
            'prediksi_peluang_2023': row['prediksi_peluang_2023'],
            'prediksi_scaled_2023': row['prediksi_scaled_2023']
        })
    return rekom_list

print("=== Rekomendasi Program Studi Berdasarkan Prediksi Peluang Keterimaan Tahun 2023 ===")
print("Pilih berdasarkan:")
print("1. Nama Program Studi (Jurusan)")
print("2. Nama Universitas")
pilihan = input("Masukkan pilihan (1 atau 2): ").strip()

if pilihan == "1":
    input_prodi = input("Masukkan nama program studi: ").strip()
    hasil = rekomendasi_berdasarkan_prodi(input_prodi, top_n=10)
    if isinstance(hasil, str):
        print(hasil)
    else:
        print(f"\nRekomendasi program studi '{input_prodi}':\n")
        for r in hasil:
            print(f"{r['rank']}. {r['nama_prodi']} ({r['nama_univ']})")
            print(f"   Prediksi Peluang: {r['prediksi_peluang_2023']:.3f} | Skala: {r['prediksi_scaled_2023']:.3f}")

elif pilihan == "2":
    input_univ = input("Masukkan nama universitas: ").strip()
    hasil = rekomendasi_berdasarkan_universitas(input_univ, top_n=10)
    if isinstance(hasil, str):
        print(hasil)
    else:
        print(f"\nRekomendasi program studi di '{input_univ}':\n")
        for r in hasil:
            print(f"{r['rank']}. {r['nama_prodi']} ({r['nama_univ']})")
            print(f"   Prediksi Peluang: {r['prediksi_peluang_2023']:.3f} | Skala: {r['prediksi_scaled_2023']:.3f}")
else:
    print("Pilihan tidak valid.")

=== Rekomendasi Program Studi Berdasarkan Prediksi Peluang Keterimaan Tahun 2023 ===
Pilih berdasarkan:
1. Nama Program Studi (Jurusan)
2. Nama Universitas

Rekomendasi program studi 'Ilmu Komputer':

1. ILMU KOMPUTER (UNIVERSITAS SEMBILAN BELAS NOVEMBER KOLAKA)
   Prediksi Peluang: 0.737 | Skala: 0.900
2. ILMU KOMPUTER (UNIVERSITAS HALUOLEO)
   Prediksi Peluang: 0.516 | Skala: 0.829
3. PENDIDIKAN ILMU KOMPUTER (UNIVERSITAS PENDIDIKAN INDONESIA)
   Prediksi Peluang: 0.490 | Skala: 0.821
4. ILMU KOMPUTER (UNIVERSITAS NUSA CENDANA)
   Prediksi Peluang: 0.439 | Skala: 0.804
5. ILMU KOMPUTER (UNIVERSITAS LAMPUNG)
   Prediksi Peluang: 0.360 | Skala: 0.779
6. ILMU KOMPUTER (UNIVERSITAS ISLAM NEGERI SUMATERA UTARA)
   Prediksi Peluang: 0.345 | Skala: 0.774
7. ILMU KOMPUTER (UNIVERSITAS LAMBUNG MANGKURAT)
   Prediksi Peluang: 0.344 | Skala: 0.774
8. ILMU KOMPUTER (UNIVERSITAS GADJAH MADA)
   Prediksi Peluang: 0.264 | Skala: 0.748
9. ILMU KOMPUTER (UNIVERSITAS INDONESIA)
   Prediksi Peluang: 0.